In [ ]:
# CHECK IF DATA FROM BOTH SCANNERS HAVE THE SAME RESULTS

In [ ]:
from s2scan.s2scan import s2sr_gcerdata

scanner = s2sr_gcerdata.S2L2AScanner(
    root_folder="Z:/dbcenter/images/sentinel/scenes/level_sr/",
    ini_date='2022-01-01',
    end_date='2022-12-31',
    max_cloud_cover=40,
    tiles=['10SFG', '34HCH'],
)
bands_10m = scanner.get_files_info(resolution=10, bands=["B02", "B03", "B04", "B08"])
bands_20m = scanner.get_files_info(resolution=20, bands=["B11",])
scl_band_info = scanner.get_files_info(resolution=20, bands=["SCL",])

In [ ]:
from fieldseg import s2sr_gcerdata

scanner = s2sr_gcerdata.S2L2AScanner(
    root_folder="Z:/dbcenter/images/sentinel/scenes/level_sr/",
    ini_date='2022-01-01',
    end_date='2022-12-31',
    max_cloud_cover=40,
    tiles=['10SFG', '34HCH'],
)
bands_10m_ref = scanner.get_files_info(resolution=10, bands=["B02", "B03", "B04", "B08"])
bands_20m_ref = scanner.get_files_info(resolution=20, bands=["B11",])
scl_band_info_ref = scanner.get_files_info(resolution=20, bands=["SCL",])

In [ ]:
from fieldseg import s2sr_gcerdata, xarray_utils
from pathlib import Path
import rioxarray

In [ ]:
scl_data_20m = s2sr_gcerdata.load_tile_dataarray(
    scl_band_info, chunks={"x": 1024, "y": 1024}
)
# 10m bands
bands_data_10m = s2sr_gcerdata.load_tile_dataarray(
    bands_10m, chunks={"x": 1024, "y": 1024}
)
bands_data_10m = s2sr_gcerdata.normalize_s2(bands_data_10m, bands_10m)
# SCL is reprojected using nearest neighbor resampling to avoid creating new values.
scl_data_10m = xarray_utils.xr_reproject_match(
    scl_data_20m, bands_data_10m, resampling="nearest"
)
bands_data_10m = xarray_utils.apply_s2_scl_mask(bands_data_10m, scl_data_10m)

In [ ]:
bands_data_10m = bands_data_10m.isel(x=slice(7000, 7512), y=slice(5000, 5512))
bands_data_10m = bands_data_10m.groupby("time.month").mean().load()

In [ ]:
root_export_folder = Path('./images')
for month, month_data in bands_data_10m.groupby("month"):
    for band, band_data in month_data.groupby("band"):
        export_folder = root_export_folder / str(month)
        export_folder.mkdir(exist_ok=True, parents=True)
        file = f'month_{month}_band_{band}.tif'
        band_data.rio.to_raster(export_folder / file, compress="LZW")

In [ ]:
bands_data_10m.sel(band=["B04", "B03", "B02", ]).plot.imshow(col="month", col_wrap=4, robust=True)